<a href="https://colab.research.google.com/github/vilmarferreira86/tech-challenge-fase03-rag-gpt3.5/blob/main/RAG_gpt3_5_turbo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Etapa 1: Instalar bibliotecas
!pip install openai faiss-cpu tiktoken --quiet

In [2]:
!pip install --upgrade openai

In [26]:
# Etapa 2: Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Etapa 3: Leitura do arquivo trn.json
import json

file_path = '/content/drive/MyDrive/FIAP/fine-tuning/projeto-fase03/trn.json'

def stream_json(file_path, limit = 10000):
  with open(file_path, 'r') as f:
    for i, line in enumerate(f):
      if i >= limit:
        break
      yield json.loads(line)

data = list(stream_json(file_path))

In [31]:
from openai import OpenAI
import re
from tqdm import tqdm
import numpy as np
import faiss


client = OpenAI(api_key = 'OPENAI_API_KEY')

# Etapa 4: Realizando Embbedding com o modelo GPT 3.5 - turbo
def get_embedding(text, model="text-embedding-ada-002"):

  # Garantir que o texto seja uma string válida
  text = str(text).encode('utf-8', errors='replace').decode('utf-8')
  text = text.replace('\n', ' ').strip()
  text = re.sub(r'\s+', ' ', text)

  if not text:
    raise ValueError("Texto vazio para embedding.")

  print(f"Attempting to embed: {text[:200]}...")  # Para debug

  response = client.embeddings.create(
      input=text,
      model=model
  )
  return response.data[0].embedding

In [16]:

texts = [item['title'] for item in data if 'title' in item and item['title'].strip() != '' and 'content' in item]
contents = [item['content'] for item in data if 'title' in item and item['title'].strip() != '' and 'content' in item]

embeddings = []
for text in tqdm(texts):
    try:
        embedding = get_embedding(text)
        embeddings.append(embedding)
    except Exception as e:
        print(f"Erro ao gerar embedding para: {text[:50]}... -> {e}")

embedding_matrix = np.array(embeddings).astype('float32')

# Criar índice FAISS
index = faiss.IndexFlatL2(len(embedding_matrix[0]))
index.add(embedding_matrix)


  0%|          | 0/9999 [00:00<?, ?it/s]

Attempting to embed: Girls Ballet Tutu Neon Pink...


  0%|          | 1/9999 [00:00<1:03:33,  2.62it/s]

Attempting to embed: Adult Ballet Tutu Yellow...


  0%|          | 2/9999 [00:00<55:20,  3.01it/s]  

...

100%|█████████▉| 9996/9999 [43:27<00:00,  4.00it/s]

Attempting to embed: Dragon Keeper (Rain Wilds Chronicles, Vol. 1)...


100%|█████████▉| 9998/9999 [43:27<00:00,  4.49it/s]

Attempting to embed: Padres brillantes, maestros fascinantes: No hay jovenes dificiles, sino una educacion inadecuada (Spanish Edition)...
Attempting to embed: 212: A Novel (Ellie Hatcher)...


100%|██████████| 9999/9999 [43:27<00:00,  3.83it/s]


In [23]:
# Etapa 5: Geração de Respostas com GPT-3.5-turbo
def gerar_resposta(pergunta, top_k=10):
  # Gerar embedding da pergunta
  pergunta_embedding = np.array(get_embedding(pergunta)).astype("float32").reshape(1, -1)

  # Buscar os top_k mais similares
  D, I = index.search(pergunta_embedding, top_k)

  # Recuperar os conteúdos correspondentes
  contexto = "\n".join([f"Título: {texts[i]}\nDescrição: {contents[i]}" for i in I[0]])

  # Montar prompt
  prompt = f"""
    Você é um assistente de IA treinado com descrições de produtos da Amazon.
    Baseado nas informações abaixo, responda à pergunta do usuário de forma clara e objetiva.

    {contexto}

    Pergunta: {pergunta}
    Resposta:
    """

  # Chamada ao GPT-3.5-turbo
  response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": prompt}],
      temperature=0.7,
      max_tokens=300
  )

  return response.choices[0].message.content

In [30]:
# Etapa 6: Testar o sistema
# Exemplo de pergunta
pergunta_exemplo = "What is the description of the product Misty of Chincoteague?"
resposta = gerar_resposta(pergunta_exemplo)
print("Resposta do modelo:\n", resposta)

Attempting to embed: What is the description of the product Misty of Chincoteague?...
Resposta do modelo:
 The description of the product Misty of Chincoteague is about a young boy and girl who encounter a mysterious wild pony named Phantom on an island off the coasts of Virginia and Maryland. They are determined to capture her, leading to a frenzied roundup on Pony Penning Day. The story follows their journey with Phantom and her gentle, loyal colt Misty, revealing the opposing forces of humans and nature.
